In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome import options
import selenium
import json
from datetime import datetime
import time

SERIE_A = "https://oddspedia.com/br/futebol/brasil/brasileirao-serie-a"
SERIE_B = "https://oddspedia.com/br/futebol/brasil/brasileirao-serie-b"
LIBERTADORES = "https://oddspedia.com/br/futebol/america-do-sul/conmebol-libertadores"
API_BASE = "https://oddspedia.com/api/v1/getMatchOdds?wettsteuer=0&geoCode=BR&bookmakerGeoCode=BR&bookmakerGeoState=&language=br" 
# &matchid={matchid}&oddGroupID={odds_market}&inplay=0
# odds_market = {1, 4}; 1 = h2h; 4 = over/under

In [2]:
opt = options.Options()
opt.add_argument("--start-maximized")
driver = webdriver.Chrome(options=opt)
driver.minimize_window()

In [3]:
driver.get(SERIE_A)
games_all = {}

In [4]:
year_selector = driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div[1]/div[2]/div[2]/div/div/main/div[1]/div[2]/button')
year_selector.click()

div2022 = driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div[1]/div[2]/div[2]/div/div/main/div[1]/div[2]/div/div[2]')
div2022.click()

time.sleep(1)

weeks = (
    driver.find_element(By.XPATH, '//*[@id="match-odds"]/div/div[1]/div[1]/div')
    ).find_elements(By.CSS_SELECTOR, "*")
next_week_btn = driver.find_element(By.XPATH, '//*[@id="match-odds"]/div/div[2]/button[2]')

In [5]:
def chunk_list(lst, n):
    return [lst[i: i + n] for i in range(0, len(lst), n)]

for week in weeks:
    games_in_current_page = driver.find_elements(By.CLASS_NAME, "match-url")
    scores_in_current_page = driver.find_elements(By.CLASS_NAME, "match-score-result__score")
    teams_in_current_page = driver.find_elements(By.CLASS_NAME, "match-team__name")
    week_visualized = driver.find_element(By.XPATH, '//*[@id="match-odds"]/div/div[2]/div').text
    for i, url in enumerate(games_in_current_page):
        games_all[url.get_attribute("href").split("=")[1]] = {
            "home": teams_in_current_page[i*2].text,
            "away": teams_in_current_page[i*2 + 1].text,
            "home_score": scores_in_current_page[i*2].text,
            "away_score": scores_in_current_page[i*2 + 1].text,
            "week": week_visualized
        }
            
    try:
        next_week_btn.click()
    except selenium.common.exceptions.ElementClickInterceptedException:
        pass

print(week_visualized)

SEMANA 32
06 NOV - 12 NOV


In [6]:
def clean_odds(clean_data, full_odds, market):
    if market == "over/under":
        names = ["over", "under"]
    elif market == "spread":
        names = ["home", "away"]
    elif market == "exact":
        names = ["odd"]
    
    divisions = full_odds[market]["periods"][0]["odds"]
    odds_all = [*divisions["alternative"], *divisions["main"]]
    for odds_list_item in odds_all:
        clean_data[match_id][market][odds_list_item["name"]] = {}
        for _id, odd in odds_list_item["odds"].items():
            clean_data[match_id][market][odds_list_item["name"]][odd["bookie_name"]] = {
                n[i]: odd[f"o{i+1}"] for i, n in enumerate(names)
            }

    return

market_index = {1: 0, 3: 2, 4: 1, 8: 7}
clean_data = {}
base_url = "https://oddspedia.com/api/v1/getMatchOdds?wettsteuer=0&geoCode=BR&bookmakerGeoCode=BR&bookmakerGeoState=&matchId={match_id}&oddGroupId={odds_market}&inplay=0&language=br".format
for match_id in games_all:
    match_odds = {}
    clean_data[match_id] = {}
    for market in market_index.keys():
        ids = {1: "h2h", 3: "spread", 4: "over/under", 8: "exact"}
        clean_data[match_id][ids[market]] = {}

        driver.get(base_url(match_id = match_id, odds_market = market))
        incomplete_data = json.loads(driver.find_element(By.TAG_NAME, 'pre').text)
        
        current_market_data = incomplete_data["data"]["prematch"][market_index[market]]
        match_odds[ids[market]] = current_market_data
        

    for odd in match_odds["h2h"]["periods"][0]["odds"]:
        clean_data[match_id]["h2h"][odd["bookie_name"]] = {
            "home": odd["o1"],
            "draw": odd["o2"],
            "away": odd["o3"]
        }

    clean_odds(clean_data, match_odds, "over/under")
    clean_odds(clean_data, match_odds, "spread")
    clean_odds(clean_data, match_odds, "exact")

    break